In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

Load Model & Tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3-3B-Instruct", load_in_8bit=True, device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3-3B-Instruct")

LoRA Setup

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

Data Formatting

In [ ]:
def format_example(text, label=""):
    return f"""
### Instruction:
Classify the following text into: Neutral, Hate-Speech, or Sarcasm.

### Input:
{text}

### Output:
{label}
"""

Training Arguments & Trainer

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=50,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    max_seq_length=512,
    args=training_args,
)

Start Training

In [ ]:
trainer.train()

Evaluation Logic

In [ ]:
def predict(text):
    prompt = format_example(text)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=5)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    for label in ["Neutral", "Hate-Speech", "Sarcasm"]:
        if label.lower() in decoded.lower():
            return label
    return "Neutral"